## Importare file json del dataset AcousticBrainz come dizionari

In [12]:

import os

creiamo una lista contentente tutti in nomi dei files in una cartella specificata. In particolare ci serve avere la lista di 
tutti i files json in modo da poter itereare e automatizzare l'interrogazione dell'API del database MusicBrainz (per ottenere il valore della variabile risposta per ogni brano, ovvero il suo genere) e dell'inserimento in un dataframe pandas

In [13]:
fdir=r'C:\Users\Luca\Downloads\acousticbrainz-lowlevel-json-20150129\acousticbrainz-lowlevel-json-20150129\lowlevel\0\00'

In [14]:
files=os.listdir(fdir)
## questa funzione del modulo os ritorna una lista contentente tutti i nomi dei files, cartelle contenuti
## nel percorso specificato come argomento

Come si vede dalla descrizione al link https://acousticbrainz.org/download, il nome di ogni file rappresenta l'MBID del brano nel campo 'recording'  del database MusicBrainz (Vedere lo schema del database al link https://musicbrainz.org/doc/MusicBrainz_Database/Schema) più, alla fine, separato da una linea, il numero di duplicati che nel database sono collegati a quel particolare MBID. Perciò ci servirà togliere ai nomi l'ultimo trattino e l'ultimo numero per poter interrogare il database. Poichè sapere il numero di duplicati comunque può essere utile, teniamo salvati tali numeri trasformando la listo originale in una lista di tuple di due elementi, il primo contenente l'MBID, il secondo contenete il numero di duplicati. 

Importiamo ora il modulo 'json', che permette di importare file json come dizionari

In [15]:
import json

Importiamo uno a caso tra i file del database per avere un' idea di cosa contiene

In [16]:
with open(fdir+'\\'+files[100],'r') as file:   ## 100 è un indice a caso, il primo che mi è venuto in mente
    jfile=json.load(file)

## prima occhiata alla struttura del file

In [17]:
jfile['rhythm']['beats_position']

[0.615328788757,
 1.24226760864,
 1.88081634045,
 2.50775504112,
 3.14630389214,
 3.78485250473,
 4.41179132462,
 5.05034017563,
 5.6888885498,
 6.32743740082,
 6.9543762207,
 7.5464849472,
 8.13859367371,
 8.73070240021,
 9.33442115784,
 9.93814086914,
 10.5186395645,
 11.0875282288,
 11.656416893,
 12.2253055573,
 12.7941951752,
 13.3630838394,
 14.0132427216,
 14.6517915726,
 15.2903394699,
 15.9404983521,
 16.5558280945,
 17.1827659607,
 17.7980957031,
 18.483083725,
 19.1680717468,
 19.8298416138,
 20.4799995422,
 21.1417675018,
 21.8151473999,
 22.5465755463,
 23.2663936615,
 23.9862117767,
 24.7060317993,
 25.4258499146,
 26.1456680298,
 26.8770961761,
 27.6085262299,
 28.3283443451,
 29.0481624603,
 29.7679805756,
 30.4878005981,
 31.2076187134,
 31.9274368286,
 32.6472549438,
 33.3670730591,
 34.0752830505,
 34.7718811035,
 35.4684791565,
 36.2463493347,
 37.0242156982,
 37.755645752,
 38.4870758057,
 39.2068939209,
 39.9267120361,
 40.5768699646,
 41.2270278931,
 41.830745697

AcousticBrainz Elabora ogni brano tramite un pacchetto di algoritmi chiamato 'Essentia', Ogni campo di questi file è il risultato dell'elaborazione del brano tramite un particolare algoritmo di questo pacchetto. Per esempio il campo 'Barkbands' riporta il risultato dell'elborazione dell'energia sonora del brano in 27 particolari bande di frequenza (bande di Bark: https://en.wikipedia.org/wiki/Bark_scale); come vediamo sono presenti diversi sottocampi: 'dmean' memorizza l'energia sonora media della derivata del segnale sonoro (nelle 27 diverse bande), dmean2 fa lo stesso per la derivata seconda, etc... Per i dettagli dare un occhiata a https://essentia.upf.edu/documentation/streaming_extractor_music.html#music-descriptors

MusicBrainz Per ogni brano fornisce sia dati 'lowlevel' (quelli che abbiamo noi), ovvero il diretto risultato dell'elaborazione del brano tramite gli algoritmi 'Essentia', sia dati 'highlevel', ovvero il risultato di algoritmi che, a partire da dati lowlevel, cercano di classificare il brano in base a diverse caratteristiche , come genere, ballabilità etc.


##### l'idea per il progetto sarebbe quella di usare i dati lowlevel per stimare un modello di classificazione (analisi del determinante o albero di classificazione) 'fai da te'


Le nostre esplicative saranno quindi i dati lowlevel e la variabile risposta potrebbe essere per esmpio il genere


#### Un problema da risolvere è che il dataset è enorme (>40GB)

dovremmo decidere se usarne solo una parte o usare il calcolo parallelo

#### Un altro problema è che MusicBrainz contiene, per ogni brano, più di un genere o, talvolta, nemmeno uno

dovremo quindi decidere che genere assegnare ai brani che hanno più di un genere sul database e che fare con i brani che non ne hanno nemmeno uno


## Salvare recording_id e artist_id eliminando il resto dei metadati

In [18]:
jfile.keys()

dict_keys(['lowlevel', 'metadata', 'rhythm', 'tonal'])

Nella sezione metadata ci sono alcuni dati sugli artisti e la frequenza di campionamento del brano; vediamo:

In [19]:
jfile['metadata']

{'audio_properties': {'analysis_sample_rate': 44100,
  'bit_rate': 0,
  'codec': 'flac',
  'downmix': 'mix',
  'equal_loudness': 0,
  'length': 490.266662598,
  'lossless': True,
  'md5_encoded': '9fa534ec3ff43ab1b7d173c9f076b860',
  'replay_gain': -10.8989887238,
  'sample_rate': 44100},
 'tags': {'acoustid_id': ['f71cb240-0b5d-4744-88ca-a73f01922268'],
  'album': ['Ágætis byrjun'],
  'albumartist': ['Sigur Rós'],
  'albumartistsort': ['Sigur Rós'],
  'artist': ['Sigur Rós'],
  'artistsort': ['Sigur Rós'],
  'date': ['1999'],
  'file_name': '05 - Ný batterí.flac',
  'musicbrainz_albumartistid': ['f6f2326f-6b25-4170-b89d-e235b25508e8'],
  'musicbrainz_albumid': ['344183d6-7aae-3b68-b2cc-b24f9ca79bbf'],
  'musicbrainz_artistid': ['f6f2326f-6b25-4170-b89d-e235b25508e8'],
  'musicbrainz_recordingid': ['0003dd36-b4d2-4216-a37e-b110f6882ecb'],
  'title': ['Ný batterí'],
  'totaltracks': ['10'],
  'tracknumber': ['5'],
  'tracktotal': ['10']},
 'version': {'essentia': '2.1-beta1',
  'essenti

Gli unici dati che ci servono sono il recording_id  e l'artist_id (ci serviranno per ottenere il genere dal database MusicBrainz).
Eliminiamo tutto il resto della sezione 'metadata'

In [20]:
jfile['rec_id']=jfile['metadata']['tags']['musicbrainz_recordingid'][0]
jfile['artist_id']=jfile['metadata']['tags']['musicbrainz_artistid'][0]
del jfile['metadata']

## Eliminare 'beats_position'

Visto che la posizione esatta dei beat è un dato non facilmente confrontabile tra brani diversi e visto che si tratta di una lista lunghissima, la eliminiamo dal file

In [21]:
del jfile['rhythm']['beats_position']

## Interrogare il database MusicBrainz

importiamo il modulo 'musicbrainzngs' (che serve per interrogare il database MusicBrainz) e proviamo ad ottenere il genere per un brano a caso.
Per vedere la documentazione visitare https://python-musicbrainzngs.readthedocs.io/en/v0.6/api/

In [22]:
import musicbrainzngs as mb  

In [23]:
mb.set_useragent('Python-urllib','any')    ##queste istruzioni servono per impostare alcuni parametri di autenticazione per interrogare il database
mb.auth('luca1290','luca1290')          

In [24]:
results=mb.get_recording_by_id(jfile['rec_id'],includes=['tags','user-tags'])  ## funzione che permette di ottenere le info di un brano 
                                                                  ## contenute nel campo 'recording' del database MusicBrainz
                                                                   ## tramite 
                                                                   ## il suo mbid (che nel nostro dataset è dato dal nome
                                                                   ## del file senza il trattino ed il numero finale);
                                                                   ## con il parametro includes impostiamo quali voci vengono
                                                                   ## restituite dall'interrogazione; a noi interessano 'tag'
                                                                   ## e 'user-tags' 

In [25]:
results

{'recording': {'id': '0003dd36-b4d2-4216-a37e-b110f6882ecb',
  'title': 'Ný batterí',
  'length': '492653',
  'tag-list': [{'count': '1', 'name': 'alternative rock'},
   {'count': '2', 'name': 'experimental rock'},
   {'count': '1', 'name': 'iceland'},
   {'count': '1', 'name': 'icelandic'},
   {'count': '1', 'name': 'post rock'}]}}

Per questo brano l'interrogazione restituisce più di un genere. Vediamo che per ogni genere ci sono delle conte, ovvero il numero di volte in cui il brano è taggato in quel modo; un modo per risolvere il nostro problema potrebbe allora essere quello di prendere semplicemente il tag con la conta più alta. Per la classificazione dovremo però decidere a priori i possibili generi assegnabili ad un brano (che dovranno essere possibilmente un numero limitato). Se riusciamo a trovare sul sito di MusicBrainz una lista di tutti i possibili tag possiamo allora mappare la ista dei possibili tag alla lista dei generi che noi abbiamo deciso a priori.

Per alcuni brani invece il database non restituisce nessun tag; Come risolvere? Potremmo controllare i tag nel campo artist nel database Musicbrainz e se anche lì non sono presenti tag scartare il brano.

## Aggiunta del genere ad un brano

Ipotizziamo di aver risolto il problema di decidere quale tag assegnare ai brani che che ne hanno più di uno e di aver deciso che fare dei brani che non ne hanno nemmeno uno. 
Prendiamo in cosiderazione il brano all'indice 100 (quello usato per la prima interrogazione).Aggiungiamo il genere al file usando il tag che ha la conta maggiore

In [26]:
maxcount=max(tag['count'] for tag in results['recording']['tag-list'])

    
jfile['genere']=[tag['name'] for tag in results['recording']['tag-list'] if tag['count']==maxcount][0]

In [27]:
jfile['genere']

'experimental rock'

Ora che abbiamo ottenuto il genere il record_id e l'artist_id non ci serve più

In [28]:
del jfile['rec_id']
del jfile['artist_id']

## Codifica Del brano come riga di un dataframe pandas

il dizionario risultante dalla letture del file json è annidato e alcuno dei valori interni sono liste. Per far si che ogni elemento di tali liste divenga una colonna a se stante utilizziamo la funzione 'json_normalize()' . In questo modo ogni file json diventa un oggetto dataframe su una sola riga. Tale funzione però trasforma in nuove colonne solo gli elementi di un dizionario; se un valore è, per esempio, una lista, tale valore verrà mantenuto in un unica cella del dataframe. Dobbiamo perciò trasformare ogni lista in un dizionario.

Per capirci:

In [29]:
import pandas as pd

In [30]:
d=pd.DataFrame(jfile)
d.head(5)

,lowlevel,rhythm,tonal,genere
average_loudness,0.550233,NaN,NaN,experimental rock
barkbands,"{'dmean': [0.000100480268884, 0.00253548240289...",NaN,NaN,experimental rock
barkbands_crest,"{'dmean': 1.67212295532, 'dmean2': 2.676247358...",NaN,NaN,experimental rock
barkbands_flatness_db,"{'dmean': 0.0150233358145, 'dmean2': 0.0234245...",NaN,NaN,experimental rock
barkbands_kurtosis,"{'dmean': 5.50720024109, 'dmean2': 9.312459945...",NaN,NaN,experimental rock


vediamo che così non va perchè un brano non è su una sola riga (vogliamo che il dataset abbia unità sulle righe e variabili sulle colonne)

nemmeno usando il metodo stack() o unstack() il risultato è buono perchè alcuni elementi rimangono liste o dizionari ( noi vogliamo che per ogni eemento del dizionario ci sia una colonna diversa), inoltre molti elementi sono NaN (non so perchè)

In [31]:
d.unstack()

lowlevel  average_loudness                                                             0.550233
          barkbands                           {'dmean': [0.000100480268884, 0.00253548240289...
          barkbands_crest                     {'dmean': 1.67212295532, 'dmean2': 2.676247358...
          barkbands_flatness_db               {'dmean': 0.0150233358145, 'dmean2': 0.0234245...
          barkbands_kurtosis                  {'dmean': 5.50720024109, 'dmean2': 9.312459945...
          barkbands_skewness                  {'dmean': 0.386306405067, 'dmean2': 0.60485827...
          barkbands_spread                    {'dmean': 2.08361315727, 'dmean2': 3.286306142...
          beats_count                                                                       NaN
          beats_loudness                                                                    NaN
          beats_loudness_band_ratio                                                         NaN
          bpm                           

Vediamo allora che usando il metodo json_normalize() ci avviciniamo un po'

In [32]:
pd.io.json.json_normalize(jfile)

,genere,lowlevel.average_loudness,lowlevel.barkbands.dmean,lowlevel.barkbands.dmean2,lowlevel.barkbands.dvar,lowlevel.barkbands.dvar2,lowlevel.barkbands.max,lowlevel.barkbands.mean,lowlevel.barkbands.median,lowlevel.barkbands.min,...,tonal.hpcp_entropy.min,tonal.hpcp_entropy.var,tonal.key_key,tonal.key_scale,tonal.key_strength,tonal.thpcp,tonal.tuning_diatonic_strength,tonal.tuning_equal_tempered_deviation,tonal.tuning_frequency,tonal.tuning_nontempered_energy_ratio
0,experimental rock,0.550233,"[0.000100480268884, 0.00253548240289, 0.002584...","[0.000161465461133, 0.00444418611005, 0.004282...","[1.45735654655e-07, 6.86095372657e-05, 3.69072...","[3.71625134221e-07, 0.000208212339203, 0.00010...","[0.00777646200731, 0.108975812793, 0.095311954...","[0.000115350965643, 0.00468042818829, 0.006785...","[8.57136910781e-07, 0.00022918661125, 0.004938...","[3.85919851386e-25, 2.91708180419e-24, 2.86548...",...,0,0.644983,B,minor,0.745978,"[1, 0.796665012836, 0.548127174377, 0.84051418...",0.524532,0.121344,438.731049,0.82062


Ci siamo quasi ma come si vede le liste rimangono liste; perchè ogni elemnto di tali liste venga trasformato in una colonna serve trasformarle in dizionari 

ispezionare a mano la struttura dei file è piuttosto laborioso. Scriviamo una funzione che ci restituisca una lista di liste in cui ogni tupla rappresenti le chiavi dei valori che non sono atomici ma sono liste


In [33]:
def klist(f,l=[]):
    li=[]
    if type(f) is list:
        if type(f[0]) is list:
            return 'll'
        return 'l'
    elif type(f) is dict:
        for k,v in f.items():
            t=klist(v,l+[k])
            if t=='l':
                li.append(l+[k])
            elif t=='ll':
                li.append(l+[k,'l'])  
            elif type(t) is list:
                for i in t:
                    li.append(l+i)
        return li
    else:
        return 'altro'
            
           


In [34]:
t=klist(jfile)
t                                 ### il primo indice è ripetuto due volte e va ignorato (solo per le sottoliste di
                                  ### lunghezza 4), dovrei migliorare la funzione.

[['lowlevel', 'lowlevel', 'barkbands', 'dmean'],
 ['lowlevel', 'lowlevel', 'barkbands', 'dmean2'],
 ['lowlevel', 'lowlevel', 'barkbands', 'dvar'],
 ['lowlevel', 'lowlevel', 'barkbands', 'dvar2'],
 ['lowlevel', 'lowlevel', 'barkbands', 'max'],
 ['lowlevel', 'lowlevel', 'barkbands', 'mean'],
 ['lowlevel', 'lowlevel', 'barkbands', 'median'],
 ['lowlevel', 'lowlevel', 'barkbands', 'min'],
 ['lowlevel', 'lowlevel', 'barkbands', 'var'],
 ['lowlevel', 'lowlevel', 'erbbands', 'dmean'],
 ['lowlevel', 'lowlevel', 'erbbands', 'dmean2'],
 ['lowlevel', 'lowlevel', 'erbbands', 'dvar'],
 ['lowlevel', 'lowlevel', 'erbbands', 'dvar2'],
 ['lowlevel', 'lowlevel', 'erbbands', 'max'],
 ['lowlevel', 'lowlevel', 'erbbands', 'mean'],
 ['lowlevel', 'lowlevel', 'erbbands', 'median'],
 ['lowlevel', 'lowlevel', 'erbbands', 'min'],
 ['lowlevel', 'lowlevel', 'erbbands', 'var'],
 ['lowlevel', 'lowlevel', 'gfcc', 'cov', 'l'],
 ['lowlevel', 'lowlevel', 'gfcc', 'icov', 'l'],
 ['lowlevel', 'lowlevel', 'gfcc', 'mean'],
 

Le sottoliste che terminano con una 'l' rappresentano il percorso per liste che hanno a loro volta delle sottoliste. Esaminiamole:

Vediamo che le uniche liste che hanno al loro interno delle sottoliste sono, nel campo lowlevel, la matrice di covarianza ('cov') e l'inversa della matrice di covarianza ('icov') dell'mfcc e del gfcc. Scopriamo cosa sono:

https://essentia.upf.edu/documentation/reference/streaming_MFCC.html 

https://en.wikipedia.org/wiki/Mel-frequency_cepstrum



https://essentia.upf.edu/documentation/reference/std_GFCC.html

Potremmo decidere di utilizzare una misura di sintesi di queste matrici (per esempio il determinante) e di tenere solo una tra cov e icov (una è l'inversa dell'altra)

Ipotizziamo di eliminare le matrici inverse e si tenere solo il determinante di quelle non inverese:

In [35]:
import numpy as np
 

np.linalg.det(jfile['lowlevel']['gfcc']['cov'])

1.7778320446877566e+32

Vediamo che il determinante è un numero enorme, questo potrebbe creare problemi durante la stima del modello. Forse allora meglio tenere il determinante della atrice di correlazione

In [36]:
vardiag_inv=np.diag(np.diag(jfile['lowlevel']['gfcc']['cov'])**(-0.5))
jfile['lowlevel']['gfcc']['cov']=vardiag_inv.dot(np.array(jfile['lowlevel']['gfcc']['cov']).dot(vardiag_inv))

In [37]:
np.linalg.det(jfile['lowlevel']['gfcc']['cov'])

0.0025263223454256014

molto meglio!....


In [38]:
jfile['lowlevel']['gfcc']['cov']=np.linalg.det(jfile['lowlevel']['gfcc']['cov'])

ora facciamo la stessa cosa per l'mfcc ed eliminiamo le rispettive inverse

In [39]:
vardiag_inv=np.diag(np.diag(jfile['lowlevel']['mfcc']['cov'])**(-0.5))
jfile['lowlevel']['mfcc']['cov']=vardiag_inv.dot(np.array(jfile['lowlevel']['mfcc']['cov']).dot(vardiag_inv))
jfile['lowlevel']['mfcc']['cov']=np.linalg.det(jfile['lowlevel']['mfcc']['cov'])
jfile['lowlevel']['mfcc']['cov']

0.0028130636864268834

Vediamo che sono quesi identici....in effetti l'mfcc e il gfcc sono molto simili....potremo magari decidere di eliminare uno dei due in toto...per il momento teniamoli entrambi.

Eliminiamo le matrici inverse:

In [40]:
del jfile['lowlevel']['gfcc']['icov']
del jfile['lowlevel']['mfcc']['icov']

Bene ora riusiamo la funzione klist() per otenere gli indici delle liste rimanenti

In [41]:
t=klist(jfile)
t

[['lowlevel', 'lowlevel', 'barkbands', 'dmean'],
 ['lowlevel', 'lowlevel', 'barkbands', 'dmean2'],
 ['lowlevel', 'lowlevel', 'barkbands', 'dvar'],
 ['lowlevel', 'lowlevel', 'barkbands', 'dvar2'],
 ['lowlevel', 'lowlevel', 'barkbands', 'max'],
 ['lowlevel', 'lowlevel', 'barkbands', 'mean'],
 ['lowlevel', 'lowlevel', 'barkbands', 'median'],
 ['lowlevel', 'lowlevel', 'barkbands', 'min'],
 ['lowlevel', 'lowlevel', 'barkbands', 'var'],
 ['lowlevel', 'lowlevel', 'erbbands', 'dmean'],
 ['lowlevel', 'lowlevel', 'erbbands', 'dmean2'],
 ['lowlevel', 'lowlevel', 'erbbands', 'dvar'],
 ['lowlevel', 'lowlevel', 'erbbands', 'dvar2'],
 ['lowlevel', 'lowlevel', 'erbbands', 'max'],
 ['lowlevel', 'lowlevel', 'erbbands', 'mean'],
 ['lowlevel', 'lowlevel', 'erbbands', 'median'],
 ['lowlevel', 'lowlevel', 'erbbands', 'min'],
 ['lowlevel', 'lowlevel', 'erbbands', 'var'],
 ['lowlevel', 'lowlevel', 'gfcc', 'mean'],
 ['lowlevel', 'lowlevel', 'melbands', 'dmean'],
 ['lowlevel', 'lowlevel', 'melbands', 'dmean2'],

Ora in t troviamo quindi le chiavi di ogni elemento di jfile il cui tipo è una lista semplice.

dobbiamo trasformare ogni lista in un dizionario

In [42]:

for i in t:
    if len(i)==2:
        jfile[i[0]][i[1]]={str(k):v for k,v in enumerate(jfile[i[0]][i[1]])}
    else:
        jfile[i[1]][i[2]][i[3]]={str(k):v for k,v in enumerate(jfile[i[1]][i[2]][i[3]])}

vediamo se ha funzionato:


In [43]:
klist(jfile)

[]

bene, ora tutte quelle che erano liste sono dizionari. Possiamo usare json_normalize() per ottenere dal file una riga di un dataframe 

In [44]:
riga1=pd.io.json.json_normalize(jfile)
riga1

,genere,lowlevel.average_loudness,lowlevel.barkbands.dmean.0,lowlevel.barkbands.dmean.1,lowlevel.barkbands.dmean.10,lowlevel.barkbands.dmean.11,lowlevel.barkbands.dmean.12,lowlevel.barkbands.dmean.13,lowlevel.barkbands.dmean.14,lowlevel.barkbands.dmean.15,...,tonal.thpcp.4,tonal.thpcp.5,tonal.thpcp.6,tonal.thpcp.7,tonal.thpcp.8,tonal.thpcp.9,tonal.tuning_diatonic_strength,tonal.tuning_equal_tempered_deviation,tonal.tuning_frequency,tonal.tuning_nontempered_energy_ratio
0,experimental rock,0.550233,0.0001,0.002535,0.000114,0.00014,0.000079,0.000138,0.000095,0.000088,...,0.526845,0.114557,0.103478,0.116441,0.197021,0.428129,0.524532,0.121344,438.731049,0.82062


le colonne sono diventate 3156 (!!!!!!) ma abbiamo circa 600000 unità quindi non è un problema.

Chiaramente quanto fatto finora andrà fatto per ogni file. Il problema è appunto che sono circa 600000 e quindi la memoria centrale si saturerà ben prima di aver finito. Andrà quindi diviso il lavoro in tanti sotto-dataset e ognuno andrà salvato su disco. Per fortuna l'analisi del discriminante si presta bene a calcolo parallelo e stime iterative quindi ce la dovremmo fare. 

## accorpare due righe di un dataframe in unico dataframe

In [315]:
pd.concat([riga1,riga1],ignore_index=True)  ##concat() impila due DataFrame; l'argomento Ignore_index, se True, reinizializza l'indice
                                    ## del dataframe risultante ignorando quelli dei dataframe originali [se non l'avessomo posto
                                    ## uguale a True in questo caso avremmo due zeri ad indicizzare le righe]

,artist_id,genere,lowlevel.average_loudness,lowlevel.barkbands.dmean.0,lowlevel.barkbands.dmean.1,lowlevel.barkbands.dmean.10,lowlevel.barkbands.dmean.11,lowlevel.barkbands.dmean.12,lowlevel.barkbands.dmean.13,lowlevel.barkbands.dmean.14,...,tonal.thpcp.4,tonal.thpcp.5,tonal.thpcp.6,tonal.thpcp.7,tonal.thpcp.8,tonal.thpcp.9,tonal.tuning_diatonic_strength,tonal.tuning_equal_tempered_deviation,tonal.tuning_frequency,tonal.tuning_nontempered_energy_ratio
0,f6f2326f-6b25-4170-b89d-e235b25508e8,experimental rock,0.550233,0.0001,0.002535,0.000114,0.00014,0.000079,0.000138,0.000095,...,0.526845,0.114557,0.103478,0.116441,0.197021,0.428129,0.524532,0.121344,438.731049,0.82062
1,f6f2326f-6b25-4170-b89d-e235b25508e8,experimental rock,0.550233,0.0001,0.002535,0.000114,0.00014,0.000079,0.000138,0.000095,...,0.526845,0.114557,0.103478,0.116441,0.197021,0.428129,0.524532,0.121344,438.731049,0.82062


## preprocessare l'intera cartella

In [50]:
len(files)

7174

In [47]:
D=pd.DataFrame()

In [48]:
rec_errlist=[]
art_errlist=[]
notaglist=[]
structerr=[]

In [52]:
for file in files[:1000]:
    
    with open(fdir+'\\'+file,'r') as f:   
        jfile=json.load(f)

    ## gen indica se il genere verrà preso direttamente dal file o se verrà effettuata un'interrogazione        
    ## infatti alcuni dei file hanno il genere nella sezione metadata, altri no

    gen='file'    
    
    
    
    ## se la chiave 'genere' non è presente nel dizionario si  interrogherà il database per ottenerlo
    try:
        jfile['genere']=jfile['metadata']['tags']['genre'][0]   
    except:                                                     
        gen='int'

        
        
    ## con il prossimo if si controlla che il genere, anche se presente come chiave, non sia una stringa vuota o una stringa 
    ##di spazi
        
    if gen=='file' and len(jfile['genere'].replace(' ',''))==0:    
        gen='int'                                                  
       

    
    ## qui inzia il blocco eseguito solo se è necessario ottenere il genere con un interrogazone del database
    if gen=='int':
        
        ##alcuni files non hanno l'etichetta 'musicbrainz_recordingid' in ['metadata']['tags'] perciò
        ## si usa try except in caso ci sia un KeyError
        try:
            jfile['rec_id']=jfile['metadata']['tags']['musicbrainz_recordingid'][0]  
        except:
            
            ## se si presenta un errore si prova comunque ad accedere a 'musicbrainz_artistid'; se c'è di nuovo errore
            ## il genere non è recuperabile in nessun modo e si passa al file successivo con continue
            try:
                jfile['artist_id']=jfile['metadata']['tags']['musicbrainz_artistid'][0]
            except:
                continue
              
        inter='r'     ### variabile che ci dice se i risultati sono presi dal campo 'recording' o dal campo 'artist'
        
        
        ## si prova ad interrogare il database tramite il recording_id; se c'è un errore (perchè nel punto precedente
        ## non si è potuto registrare il recording_id ma solo l'artist_id oppure perchè c'è un errore di connessione)
        ## si prova interrogando tramite artist_id
        try:
            results=mb.get_recording_by_id(jfile['rec_id'],includes=['tags','user-tags'])
        except:
            rec_errlist.append(file)
            ## se c'è stato errore si prova interroganto tramite artist_id; se si ha successo
            ## la variabile inter assume valore 'a'. Se c'è di nuovo errore il genere non è
            ## in alcun modo recuperabile e si passa al file successivo
            try:
                results=mb.get_artist_by_id(jfile['artist_id'],includes=['tags','user-tags'])
                inter='a'
            except:
                art_errlist.append(file)
                continue
                
        ## di seguito i due blocchi per il recupero del genere dai risultati dell'interrogazione; i due blocchi if 
        ## sono necessari perchè a seconda che l'interrogazione sia stata fatta al campo artist o recording, le chiavi 
        ## per ottenere i tag sono diverse
        
        ## in entrambi i blocchi si usa comunque un costrutto try except perchè in alcuni casi non è presente la chiave
        ## 'tag-list' nei risultati; in tal caso e necessario gestire l'errore
        if inter=='r':
            try:
                maxcount=max(tag['count'] for tag in results['recording']['tag-list'])
            except:
                try:
                    results=mb.get_artist_by_id(jfile['artist_id'],includes=['tags','user-tags'])
                    inter='a'
                except:
                    art_errlist.append(file)
                    continue
        if inter=='a':
            try:
                maxcount=max(tag['count'] for tag in results['artist']['tag-list'])
            except:
                notaglist.append(file)
                continue
            
            
        if inter=='r':
            jfile['genere']=[tag['name'] for tag in results['recording']['tag-list'] if tag['count']==maxcount][0]
        else:
            jfile['genere']=[tag['name'] for tag in results['artist']['tag-list'] if tag['count']==maxcount][0]
        
    
        try:
            del jfile['rec_id']
        except:
            pass
        try:
            del jfile['artist_id']
        except:
            pass
    
    try:
        del jfile['rhythm']['beats_position']
    except:
        pass
    try:
        del jfile['metadata']
    except:
        pass
    
    vardiag_inv=np.diag(np.diag(jfile['lowlevel']['gfcc']['cov'])**(-0.5))
    jfile['lowlevel']['gfcc']['cov']=vardiag_inv.dot(np.array(jfile['lowlevel']['gfcc']['cov']).dot(vardiag_inv))
    jfile['lowlevel']['gfcc']['cov']=np.linalg.det(jfile['lowlevel']['gfcc']['cov'])
    
    vardiag_inv=np.diag(np.diag(jfile['lowlevel']['mfcc']['cov'])**(-0.5))
    jfile['lowlevel']['mfcc']['cov']=vardiag_inv.dot(np.array(jfile['lowlevel']['mfcc']['cov']).dot(vardiag_inv))
    jfile['lowlevel']['mfcc']['cov']=np.linalg.det(jfile['lowlevel']['mfcc']['cov'])
        
    del jfile['lowlevel']['gfcc']['icov']
    del jfile['lowlevel']['mfcc']['icov']  
    
    t=klist(jfile)
    
    for i in t:
        if len(i)==2:
            jfile[i[0]][i[1]]={str(k):v for k,v in enumerate(jfile[i[0]][i[1]])}
        else:
            jfile[i[1]][i[2]][i[3]]={str(k):v for k,v in enumerate(jfile[i[1]][i[2]][i[3]])} 
    
    if len(klist(jfile))!=0:
        structerr.append(file)
        print('errore di struttura')
        continue
        
    D=pd.concat([D,pd.io.json.json_normalize(jfile)],ignore_index=True) 
    


In [53]:
D

,genere,lowlevel.average_loudness,lowlevel.barkbands.dmean.0,lowlevel.barkbands.dmean.1,lowlevel.barkbands.dmean.10,lowlevel.barkbands.dmean.11,lowlevel.barkbands.dmean.12,lowlevel.barkbands.dmean.13,lowlevel.barkbands.dmean.14,lowlevel.barkbands.dmean.15,...,tonal.thpcp.4,tonal.thpcp.5,tonal.thpcp.6,tonal.thpcp.7,tonal.thpcp.8,tonal.thpcp.9,tonal.tuning_diatonic_strength,tonal.tuning_equal_tempered_deviation,tonal.tuning_frequency,tonal.tuning_nontempered_energy_ratio
0,Progressive Metal,0.555690,2.010551e-04,0.003135,0.000080,0.000139,0.000119,0.000123,0.000087,0.000076,...,0.146471,0.122225,0.193997,0.240032,0.187197,0.163846,0.599661,0.223028,447.432556,0.881616
1,Progressive Metal,0.699075,2.147968e-07,0.000039,0.000313,0.000150,0.000133,0.000069,0.000059,0.000065,...,0.059846,0.266527,0.474299,0.321649,0.255085,0.363494,0.583240,0.014894,440.254242,0.654346
2,folk rock,0.955147,2.960552e-04,0.017483,0.000392,0.000236,0.000267,0.000155,0.000330,0.000382,...,0.291372,0.283323,0.332383,0.326154,0.243283,0.249484,0.594181,0.238814,434.193115,0.903035
3,Folk Rock,0.948380,2.999458e-04,0.017424,0.000397,0.000239,0.000267,0.000155,0.000326,0.000383,...,0.307786,0.298695,0.337496,0.316717,0.223026,0.237169,0.589065,0.224281,434.193115,0.900990
4,Folk Metal,0.955269,1.994583e-02,0.037492,0.000740,0.000680,0.000399,0.000344,0.000573,0.000399,...,0.102191,0.106128,0.159862,0.212134,0.222574,0.217372,0.512689,0.151277,434.193115,0.766163
5,Goregrind,0.953525,5.188964e-05,0.009402,0.000240,0.001150,0.000631,0.000355,0.000319,0.000382,...,0.697486,0.700704,0.796108,0.892605,0.892733,0.826382,0.329612,0.280563,455.516602,0.909796
6,Rap & Hip-Hop,0.935944,8.142671e-03,0.012766,0.000246,0.000347,0.000217,0.000191,0.000169,0.000109,...,0.214589,0.270341,0.452101,0.567680,0.487333,0.378446,0.453975,0.189041,434.193115,0.842412
7,Hip-Hop,0.950654,1.167876e-03,0.010621,0.000341,0.000394,0.000305,0.000211,0.000170,0.000233,...,0.197115,0.191539,0.194204,0.179475,0.313899,0.546419,0.568002,0.054331,434.193115,0.699958
8,Rock,0.902541,1.380009e-05,0.001554,0.000638,0.001007,0.000811,0.000404,0.000381,0.000360,...,0.175107,0.340962,0.570642,0.510715,0.536640,0.633269,0.547593,0.025623,442.037933,0.675848
9,Space Rock,0.962707,3.564815e-04,0.003175,0.000771,0.000391,0.000249,0.001436,0.000448,0.000183,...,0.130620,0.180827,0.171923,0.110972,0.073242,0.090436,0.523670,0.056967,438.984558,0.659584


In [450]:
len(set(D['genere']))

338

In [441]:
print(rec_errlist)
print(art_errlist)
print(notaglist)
print(structerr)

['0002c3c9-d635-455c-91a6-b0082b99a91c-0.json', '001d74c2-62c6-49cd-ab5f-c9c5735fc427-5.json']
['00005ac4-210c-4914-89ba-6279ea881809-0.json', '0000975b-110a-428a-b329-5207d4eb675f-0.json', '000165dd-88fe-4d05-8893-a81b7759879e-0.json', '00018ca8-d84c-4c3d-bcab-5f9a4b9e4e92-2.json', '00020316-d32d-49c2-8ea3-2177c62de66b-0.json', '00025329-c0fd-408d-bc0d-495f8adaee15-0.json', '000253f0-8c15-47eb-b805-372f68e7be18-0.json', '00026127-dbca-4547-b1be-9b5a0b31db8f-0.json', '000291b7-cc80-440b-a0c7-3d7c1fc0bc22-0.json', '0002a018-b58c-41a3-b8e8-1aa8a75fe4ed-0.json', '0002a454-035a-45e3-a74f-6f4a3190e285-0.json', '00031f0c-9120-4fe0-aea2-151d99e889f6-0.json', '0003215f-fdc6-4ff0-a209-b2d8c8d5ee1c-0.json', '00033b82-6f72-42f3-afac-c8b5c99434a2-0.json', '00033d7e-868c-4b30-b1eb-321740c07bc2-0.json', '000395dc-b6a0-44d2-bb65-ccc037e7f225-0.json', '0003a25f-3e18-474f-b55d-f7379678bd77-0.json', '0003a25f-3e18-474f-b55d-f7379678bd77-2.json', '0003af30-f566-4d13-91b6-b9c84884d69a-0.json', '00005ac4-2

In [ ]:
rec_errlist=[]
art_errlist=[]
notaglist=[]
structerr=[]

In [480]:
D1=pd.DataFrame([{'a':[1,2],'b':2},{'a':1,'c':2}])
D2=pd.DataFrame([{'a':1,'d':2}])

In [482]:
D1.dtypes

a     object
b    float64
c    float64
dtype: object

In [54]:
[i for i in D.columns if any(D[i].isnull())]

[]

In [494]:
D['tonal.key_scale']

0      major
1      minor
2      minor
3      minor
4      major
5      major
6      major
7      major
8      minor
9      major
10     minor
11     minor
12     minor
13     minor
14     minor
15     major
16     major
17     major
18     major
19     major
20     major
21     major
22     minor
23     minor
24     major
25     minor
26     minor
27     major
28     major
29     minor
       ...  
888    minor
889    major
890    major
891    minor
892    major
893    major
894    major
895    minor
896    major
897    major
898    major
899    major
900    major
901    major
902    major
903    minor
904    major
905    minor
906    major
907    major
908    minor
909    minor
910    minor
911    major
912    major
913    major
914    major
915    minor
916    major
917    major
Name: tonal.key_scale, Length: 918, dtype: object

In [477]:
type(D.iloc[:,1].iloc[0])

numpy.float64

In [479]:
np.list

AttributeError: module 'numpy' has no attribute 'list'